In [ ]:
curl "https://api.github.com/search/repositories?q=language:python+stars:>100&per_page=100&page=1"

curl "https://api.github.com/search/repositories?q=language%3Apython%2Bstars%3A15921&per_page=100&page=1"




In [8]:
import requests
import pandas as pd
import time
import random

# Constants
GITHUB_API_URL = "https://api.github.com"
PER_PAGE = 100  # Maximum allowed by GitHub API
TOTAL_REPOS = 200  # Total repositories you want to retrieve
ACCESS_TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")  # Replace with your GitHub personal access token or set to None

# Headers for authentication (optional but recommended)
headers = {}
import os

if ACCESS_TOKEN:
    headers = {"Authorization": f"token {ACCESS_TOKEN}"}

# Function to get random repositories
def get_random_repos(total_repos):
    repos_data = []
    readme_errors = []
    repos_processed = 0
    while repos_processed < total_repos:
        # Generate random stars count to randomize search results
        random_stars = random.randint(0, 50000)
        query = f"language:python+stars:{random_stars}"

        # Generate random page number
        random_page = 1#random.randint(1, 4)
        params = {
            "q": query,
            #"sort": "stars",
            #"order": "desc",
            "per_page": PER_PAGE,
            "page": random_page
        }

        response = requests.get(f"{GITHUB_API_URL}/search/repositories", headers=headers, params=params)

        if response.status_code != 200:
            print(response.json())
            print(f"Failed to fetch repositories: {response.status_code}")
            break
        print(f"Fetching page {random_page} with {random_stars} stars...")

        items = response.json().get("items", [])
        if not items:
            continue

        for repo in items:
            repo_id = repo["id"]

            # Fetch the README
            readme_url = f"{GITHUB_API_URL}/repos/{repo['full_name']}/readme"
            readme_resp = requests.get(readme_url, headers=headers, params={"accept": "application/vnd.github.v3.raw"})

            if readme_resp.status_code == 200:
                # Save README to a file named after the repo ID
                with open(f"readmes/{repo_id}.md", "w", encoding="utf-8") as f:
                    f.write(readme_resp.text)
            else:
                readme_errors.append((repo_id, readme_resp.status_code))
                continue  # Skip repositories without README

            # Collect repository data
            repo_data = {
                "repo_id": repo_id,
                "repo_name": repo["name"],
                "full_name": repo["full_name"],
                "owner_login": repo["owner"]["login"],
                "repo_url": repo["html_url"],
                "description": repo["description"],
                "primary_language": repo["language"],
                "topics": repo.get("topics", []),
                "license": repo["license"]["name"] if repo["license"] else None,
                "created_at": repo["created_at"],
                "updated_at": repo["updated_at"],
                "pushed_at": repo["pushed_at"],
                "size": repo["size"],
                "stargazers_count": repo["stargazers_count"],
                "watchers_count": repo["watchers_count"],
                "forks_count": repo["forks_count"],
                "open_issues_count": repo["open_issues_count"],
                "default_branch": repo["default_branch"],
                "score": repo["score"],
                "is_fork": repo["fork"],
                "visibility": repo.get("visibility", "public")
            }
            repos_data.append(repo_data)
            repos_processed += 1

            if repos_processed >= total_repos:
                break

        # Sleep to respect API rate limits
        time.sleep(10)

    return repos_data, readme_errors

# Main execution
if __name__ == "__main__":
    import os

    # Create directory for README files
    os.makedirs("readmes", exist_ok=True)

    # Get random repositories
    repos_data, readme_errors = get_random_repos(TOTAL_REPOS)

    # Save repository data to a DataFrame
    df = pd.DataFrame(repos_data)

    # Save DataFrame to CSV
    df.to_csv("repositories_data.csv", index=False)

    # Report any repositories where README could not be fetched
    if readme_errors:
        print("Repositories where README could not be fetched:")
        for error in readme_errors:
            print(f"Repo ID: {error[0]}, Status Code: {error[1]}")

    print(f"Successfully processed {len(df)} repositories.")

KeyboardInterrupt: 

In [3]:
response

NameError: name 'response' is not defined